In [1]:
# Importing libraries for web scraping, mainly using Selenium & BeautifulSoup
# Extracting additional info with CoinGecko API
from selenium import webdriver
# from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time, requests, json
from datetime import datetime

In [ ]:
def check_value(x):
    if 'N/A' in x:
        x = None
    return x

In [ ]:
# To try finding social media of token by name
# Sometimes sites load but return account does not exist
def get_social_media_profiles(username):
    urls = {
    'Facebook': f'https://www.facebook.com/{username}',
    'Twitter': f'https://twitter.com/{username}',
    'Instagram': f'https://www.instagram.com/{username}',
    'Youtube': f'https://www.youtube.com/@{username}',
    'Telegram': f'https://www..me/{username}'
    }   # Linkedin & Discord aren't working this way
    profiles = {}
    for name, url in urls.items():
        try:
            response = requests.get(url)
            if response.status_code != 200:
                continue
            profiles [name] = url
        except:
            continue
    return profiles

name = 'Hifi Finance'
username = name.replace(" ", "")
social_media = get_social_media_profiles(username)
social_media

In [ ]:
if 'Twitter' in social_media.keys:
    twitter_f = get_twitter_followers(username)
else:
    twitter_f = None
if 'Instagram' in social_media.keys:
    insta_f = get_insta_followers(username)
else:
    insta_f = None
if 'Telegram' in social_media.keys:
    telegram_f = get_telegram_followers(username)
else:
    telegram_f = None
if 'Facebook' in social_media.keys:
    fb_f = get_fb_followers(username)
else:
    fb_f = None
if 'Youtube' in social_media.keys:
    yt = social_media['Youtube']
else:
    yt = None
links = social_media.values

In [4]:
# To get follower count on twitter
def get_twitter_followers(username):
    browser_sm = webdriver.Chrome()  
    browser_sm.get(f'https://www.x.com/{username}')  
    time.sleep(10)
    while True:
        dates = browser_sm.find_elements(By.CSS_SELECTOR, 'a[class="css-146c3p1 r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-xoduu5 r-1q142lx r-1w6e6rj r-9aw3ui r-3s2u2q r-1loqt21"]')
        # dates = browser_sm.find_elements(By.CSS_SELECTOR, 'time')
        tnum = len(dates)
        print(tnum)
        if tnum == 0:
            print('Restarting')
            browser_sm.get(f'https://www.x.com/{username}')  
            time.sleep(10)
        elif tnum >= 9:
            ('Done')
            break
        else:
            ('Scrolling')
            browser_sm.execute_script("window.scrollTo(0, 10000)")

    fcount = browser_sm.find_elements(By.CSS_SELECTOR, value='span[class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3"]')
    for i in range(20,30):
        # print(i.text)
        if fcount[i].text == 'Followers':
            fcount = fcount[i-1].text
            break
    print('No. of followers: ', fcount)

    print('No. of tweets collected: ', len(dates))
    texts_c = browser_sm.find_elements(By.CSS_SELECTOR, 'div[class="css-146c3p1 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim"]')
    # print(len(texts_c))
    nums = browser_sm.find_elements(By.CSS_SELECTOR, 'div[class="css-175oi2r r-18u37iz r-1h0z5md r-13awgt0"]')
    # print(len(nums))

    info = []
    for num in range(10):
        print(f'Processing tweet {num}')
        d = {}
        date = dates[num].find_element(By.CSS_SELECTOR, 'time')
        date = date.text
        gaps = len(date.split(' '))
        if gaps == 2:
            date = date + f' {datetime.now().year}'
        d['date'] = date
        print(f'Date of tweet {num+1}: {date}')

        spans = texts_c[num].find_elements(By.CSS_SELECTOR, 'span')
        text = ''
        for span in spans:
            text = text + span.text
        d['text'] = text
        print(f'Tweet {num+1}: {text}')

        if num == 0:
            comments, retweets, likes, views = nums[0], nums[1], nums[2], nums[3]
        else:
            base_index = 4 * num  # Calculate the base index for the current tweet
            comments, retweets, likes, views = nums[base_index], nums[base_index + 1], nums[base_index + 2], nums[base_index + 3]

        com_num = comments.find_elements(By.XPATH, 'button/div/div[2]/span/span/span')
        print(len(com_num))
        com_num = com_num[0].text
        d['comments_num'] = com_num
        print(f'No. of comments of tweet {num+1}: {com_num}')
        re_num = retweets.find_element(By.XPATH, 'button/div/div[2]/span/span/span').text
        d['retweets_num'] = re_num
        print(f'No. of retweets of tweet {num+1}: {re_num}')
        lik_num = likes.find_element(By.XPATH, 'button/div/div[2]/span/span/span').text
        d['likes_num'] = lik_num
        print(f'Likes of tweet {num+1}: {lik_num}')
        view_num = views.find_element(By.XPATH, 'button/div/div[2]/span/span/span').text
        d['views_num'] = view_num
        print(f'Views of tweet {num+1}: {view_num}')

        info.append(d)
        print(d)

    browser_sm.quit()
    return fcount, info

username = 'rebeccawalser'#'BreederDAO'
print(get_twitter_followers(username))

3
9
No. of followers:  42K
No. of tweets collected:  9
Processing tweet 0
Date of tweet 1: Jul 11, 2023
Tweet 1: Super honored & humbled to be named as a Top 3 Most Influential Financial Advisor but the honor truly belongs to my amazing Team that never fails to find innovative ways of getting our message about strategic tax planning and risk mitigation out to the world! Thanks @Investopedia
1
No. of comments of tweet 1: 3


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"button/div/div[2]/span/span/span"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6150CEEA2+31554]
	(No symbol) [0x00007FF615047ED9]
	(No symbol) [0x00007FF614F0872A]
	(No symbol) [0x00007FF614F58434]
	(No symbol) [0x00007FF614F5853C]
	(No symbol) [0x00007FF614F4BBAC]
	(No symbol) [0x00007FF614F7D06F]
	(No symbol) [0x00007FF614F4BA76]
	(No symbol) [0x00007FF614F7D240]
	(No symbol) [0x00007FF614F9C977]
	(No symbol) [0x00007FF614F7CDD3]
	(No symbol) [0x00007FF614F4A33B]
	(No symbol) [0x00007FF614F4AED1]
	GetHandleVerifier [0x00007FF6153D8B1D+3217341]
	GetHandleVerifier [0x00007FF615425AE3+3532675]
	GetHandleVerifier [0x00007FF61541B0E0+3489152]
	GetHandleVerifier [0x00007FF61517E776+750614]
	(No symbol) [0x

In [ ]:

    # tweets = browser_sm.find_elements(By.CSS_SELECTOR, 'div.css-175oi2r[data-testid="cellInnerDiv"]')
    # print(len(tweets))
    # articles = browser_sm.find_elements(By.CSS_SELECTOR, value='article')
    # print(len(articles))
    # idivs = browser_sm.find_elements(By.CSS_SELECTOR, 'div[class="css-175oi2r r-16y2uox r-1wbh5a2 r-1ny4l3l"]')
    # print(len(idivs))
    # for div in idivs:
    #     # outer = div.find_elements(By.CSS_SELECTOR, value='div')[1]
    #     # inner = div.find_elements(By.CSS_SELECTOR, value='div')[1]
    #     inner = div.find_element(By.XPATH, './/div[2]/*[2]')
    #     tweet_divs = inner.find_elements(By.CSS_SELECTOR, value='div')
    #     print(len(tweet_divs))
    #     date_c = tweet_divs[0]
    #     text_c = tweet_divs[1]
    #     if len(tweet_divs) == 4:
    #         interaction_c = tweet_divs[3]
    #     else:
    #         interaction_c = tweet_divs[2]

In [8]:
# To get follower count on telegram
def get_telegram_followers(username):
    html = requests.get(f'https://t.me/{username}').text
    soup = BeautifulSoup(html).find('div', class_='tgme_page_extra')
    if soup == None:
        box = None
    else:
        box = soup.text[0:box.find(' mem')]
    return box

username = 'pikamo4je6wcial'
print(get_telegram_followers(username))

None


In [ ]:
# To get follower count on insta
def get_insta_followers(username):
    browser_sm = webdriver.Firefox()  
    browser_sm.get(f'https://www.instagram.com/{username}')  
    time.sleep(5)
    count = browser_sm.find_elements(By.CSS_SELECTOR, value='span[class="html-span xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x1hl2dhg x16tdsg8 x1vvkbs"]')
    check = len(count)
    if check > 0:
        count = count[1].text
    else: 
        count = None
    browser_sm.quit()
    return count

username = 'ujmfed'
print(get_insta_followers(username))

In [19]:
# To get follower count on facebook
def get_fb_followers(username):
    browser_sm = webdriver.Firefox()  
    browser_sm.get(f'https://www.facebook.com/{username}')  
    time.sleep(10)
    count = browser_sm.find_elements(By.XPATH, "//a[contains(@href, 'followers')]")
    # count = browser_sm.find_elements(By.CSS_SELECTOR, value='a[class="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1sur9pj xkrqix3 xi81zsa x1s688f"]')
    check = len(count)
    if check > 0:
        count = count[0].text.split(' ')[0]
    else:
        count = None
    browser_sm.quit()
    return count

username = 'eTukTukio'
print(get_fb_followers(username))

9.3K


In [ ]:
def get_range(num):
    options = ['0 - 1K', '1K-5K', '5K-20K', '20K-100K', 'greater than 100K']
    if num.endswith('K'):
        num = float(num[:-1])
        if num <= 1.0:
            option = options[0]
        elif num <= 5.0:
            option = options[1]
        elif num <= 20.0:
            option = options[2]
        elif num <= 100.0:
            option = options[3]
        else:
            option = options[4]
    else:
        num = int(num)
        if num <= 1000:
            option = options[0]
        elif num <= 5000:
            option = options[1]
        elif num <= 20000:
            option = options[2]
        elif num <= 100000:
            option = options[3]
        else:
            option = options[4]
    return option

print(get_range('9.3K'))


In [22]:
# To try linktree of token by name
def get_linktree(username):
    url = f'https://linktr.ee/{username}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            link = url
        else:
            link = None
    except: 
        link = None
    return link

username = 'etuktuk'#'MarginlyUpdatesandOracleFeature'
linktree = get_linktree(username)
linktree

'https://linktr.ee/etuktuk'

In [1]:
# html = requests.get('https://www.youtube.com/@etuktuk')
# print(html.status_code)
# print(html.content)
# soup = BeautifulSoup(html)
# boxes = soup.find('a', class_='x1ixi81zsa x1s688f').text
# boxes[0:boxes.find(' mem')]

In [14]:
username = 'ZeroNameService(ZNS)'
print(username)
print(username.find('('))
if username.find('(') != -1: username = username[0:name.find('(')]
if username.find('(') != -1: username = username[0:name.find('(')]
print(username)
if username.find('-') != -1: username = username[0:name.find(' -')].strip()

Zero Name Service (ZNS)
ZeroNameService(ZNS)
15
ZeroNameService(ZN


In [4]:
import pandas as pd
df = pd.read_csv(r"C:\Users\anush\OneDrive\Desktop\Work\smallcap\scraper-repo\selenium-twitter-scraper\tweets\2024-07-15_12-01-17_tweets_1-10.csv")
for i, row in df.iterrows():
    print(row[4])
    print('----------')

Yesterday on  with  we discussed the spending habits of our current administration and what must be done to bring the American economy back to good health. Watch the segment linked below!
----------
Always love joining  on ! Today we discussed Powell's testimony and whether or not we can expect a rate cut soon. You can watch the segment linked below and to learn more, visit - https://walserwealth.com. #NotYourAverageAdvisor #SchwabNetwork
----------
Ignoring the signs can cost you. Now's the time to review your finances. Plan ahead and protect your finances. #TaxIncrease #FinancialAdvice #BePrepared
----------
calling all early birds  - please join us now… with     #FoxNews #FoxBusiness
----------
A Democrat civil war is brewing…
 
Elon Musk declares war on behalf of the first amendment…
----------
Tune in to  tonight at 5:40 PM EST as I join “Carl Higbie: Frontline”!
----------
Click the link below to watch my appearance on  where we took a look at the current state of the market, wit

In [9]:
from selenium import webdriver
# from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
import time

# To get follower count on twitter
def get_twitter_followers(username):
    browser_sm = webdriver.Chrome()  
    browser_sm.get(f'https://www.x.com/{username}')  
    time.sleep(5)
    fcount = browser_sm.find_elements(By.CSS_SELECTOR, value='span[class="css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3"]')
    for i in range(20,30):
        if fcount[i].text == 'Followers':
            fcount = fcount[i-1].text
            break
    browser_sm.quit()
    return fcount

username = 'BreederDAO' #'rebeccawalser'
print('No. of followers: ', get_twitter_followers(username))

No. of followers:  132.7K
